In [176]:
import pandas as pd
import re

from maple.types import Action, ActionType, Stage
from maple.classification import parse

import warnings
warnings.filterwarnings('ignore')


In [181]:
df = pd.read_csv("maple/predictions.csv")

df['label'].fillna('--', inplace=True)
df['action'] = df['action'].str.replace(r'[HS]\d+', '[billnum]')

df.sort_values('label', inplace=True)

# df[df.label.str.contains('--')]

df.groupby(['label', 'action']).count()
# count = show.groupby(['same', 'prediction', 'label']).count()

# display(show[show['same'] == True].groupby(['action', 'prediction', 'label']).count())

action_id  \
label                   action                                                
--                      Accepted, in concurrence                          1   
                        Accompanied [billnum]                           372   
                        Accompanied [billnum] and [billnum]               2   
                        Accompanied a new draft, see [billnum]          785   
                        Accompanied a study order, see [billnum]       1899   
...                                                                     ...   
text_of_amendment       Text of amendment, see [billnum]                  2   
                        Text of an amendment, see [billnum]              22   
text_printed_as_amended Reprinted as amended, see [billnum]              32   
                        Text of [billnum], printed as amended            39   
                        Text of [billnum], reprinted as amended           1   

                                                                  prediction  
label                   action                                                
--                      Accepted, in concurrence                           1  
                        Accompanied [billnum]                            372  
                        Accompanied [billnum] and [billnum]                2  
                        Accompanied a new draft, see [billnum]           785  
                        Accompanied a study order, see [billnum]        1899  
...                                                                      ...  
text_of_amendment       Text of amendment, see [billnum]                   2  
                        Text of an amendment, see [billnum]               22  
text_printed_as_amended Reprinted as amended, see [billnum]               32  
                        Text of [billnum], printed as amended             39  
                        Text of [billnum], reprinted as amended            1  

[2985 rows x 2 columns]

In [41]:
counts = []

In [ ]:




counts.append(df.label.where(df.label == 'unc').dropna().count())

print(counts)

In [20]:

df['clean_actions'] = df['action'].str.replace(r'[AP]M','xM').str.replace(r'[,.]', '')
df['clean_actions'] = df['clean_actions'].str.replace(r'(?: ?[HS]\d+)+', ' n')
df['clean_actions'] = df['clean_actions'].str.replace(r'\d+ YEAS to \d+ NAYS \(.*$', 'vote')
df['clean_actions'] = df['clean_actions'].str.replace(r'(?:\d+ ?)+ ?(?: ?and \d+)?', 'n ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'Amendment (?:#n )?\(.*\)', 'Amendment', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'-+ ?see? Roll Call.*$', 'rollcall', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'in the ', 'in ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'as a ', 'as ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'of an ', 'of ', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'Reported from the committee on .*', 'Reported from the committee on *', regex=True)
df['clean_actions'] = df['clean_actions'].str.replace(r'The following amendments were considered as one and (\w*)', lambda x: f'The following amendments were considered as one and {x.group(1)}', regex=True)


matches = [
  (r'substituted for', 'substituted_for'), 
  ( r'^Amended', 'amended'),
  ( r'^Amendment (.*) adopted pending concurrence' , 'amended_pending')
  
]

for (match, label) in matches: 
  df['label'] = df['label'].mask(df['clean_actions'].str.contains(match), label)




display(df.groupby(['label','clean_actions']).count().sort_values('clean_actions', ascending=True).tail(30))


display(df[df['clean_actions'].str.contains(r'ordered to a third reading', flags=re.I)].groupby(['clean_actions', 'prediction', 'label']).count().sort_values('label'))


display(df.groupby(['label']).count())


action_id  \
label                   clean_actions                                                   
unc                     Signed (in part) Chapter n of the Acts of n                 1   
                        Signed (in part) by the Governor Chapter n of t...          5   
                        Signed by the Governor Chapter n of the Acts of n         246   
                        Spread upon the records of the House and sent t...          1   
                        Substitute as new text for n                                1   
substituted_as_amended  Substituted as amended as new draft for n                   3   
                        Substituted as amended as new text for n                    8   
                        Substituted as amended for n                               19   
substituted_as_new_text Substituted as anew text for n                              1   
                        Substituted as new draft for n                             12   
                        Substituted as new text for n                              17   
unc                     Substituted as new text for n                               1   
                        Substituted by a bill see n                                 1   
substituted_for         Substituted for n                                          57   
unc                     Taken out of the Notice Section                             8   
taken_out_of_od         Taken out of the Orders of the Day                        211   
unc                     Text of House adopted amendments see n                      1   
text_of_amendment       Text of a further amendment see n                           1   
                        Text of amendment see n                                    24   
text_printed_as_amended Text of n printed as amended                               39   
                        Text of n reprinted as amended                              1   
unc                     Text of the further amendment to n                          1   
amendments_considered   The following amendments were considered as one...         17   
                        The following amendments were considered as one...          2   
                        The following amendments were considered as one...         16   
                        The following amendments were considered as one...          1   
unc                     The following items and sections severally repo...          3   
reported                The following section reported from the committ...          1   
unc                     Transmitted to the Secretary of State                      10   
                        Ways and Means amendment adopted as amended                 1   

                                                                            action  \
label                   clean_actions                                                
unc                     Signed (in part) Chapter n of the Acts of n              1   
                        Signed (in part) by the Governor Chapter n of t...       5   
                        Signed by the Governor Chapter n of the Acts of n      246   
                        Spread upon the records of the House and sent t...       1   
                        Substitute as new text for n                             1   
substituted_as_amended  Substituted as amended as new draft for n                3   
                        Substituted as amended as new text for n                 8   
                        Substituted as amended for n                            19   
substituted_as_new_text Substituted as anew text for n                           1   
                        Substituted as new draft for n                          12   
                        Substituted as new text for n                           17   
unc                     Substituted as new text for n                            1   
                       

action_id  \
clean_actions                                      prediction    label                                     
Read rules suspended read second and ordered to... uncategorized multiread_second_order_third          1   
                                                                 read_second_order_third              92   
Read second and ordered to a third reading         uncategorized read_second_order_third             702   
Ordered to a third reading                         uncategorized unc                                 166   
Read second rules suspended ordered to a third ... engrossment   unc                                   1   
Read second ordered to a third reading rules su... engrossment   unc                                  19   
Read second ordered to a third reading rules su... engrossment   unc                                   1   
Read second ordered to a third reading rules su... engrossment   unc                                   1   
Read second ordered to a third reading rules su... uncategorized unc                                   1   
Read second ordered to a third reading read thi... engrossment   unc                                  25   
Read second ordered to a third reading read thi... engrossment   unc                                  21   
Read second ordered to a third reading read third  uncategorized unc                                   1   
Read second ordered to a third reading read a t... engrossment   unc                                   1   
Read second amended ordered to a third reading ... engrossment   unc                                   1   
Read second amended by substitution of a new te... uncategorized unc                                   2   
Read second amended and ordered to a third reading uncategorized unc                                   1   
Read second amended (as recommended by the comm... uncategorized unc                                   1   
Read second amended (as recommended by the comm... uncategorized unc                                   2   
Read rules suspended read second ordered to a t... engrossment   unc                                  17   
Read rules suspended read second ordered to a t... engrossment   unc                                   3   
Read rules suspended read second ordered to a t... engrossment   unc                                   1   
Ordered to a third reading rules suspended read... uncategorized unc                                   1   
Ordered to a third reading rules suspended and ... uncategorized unc                                   2   
Ordered to a third reading read third and passe... engrossment   unc                                   1   
Ordered to a third reading - vote                  uncategorized unc                                   1   
Read; rules suspended read second ordered to a ... engrossment   unc                                   1   
Red second and ordered to a third reading          uncategorized unc                                   1   

                                                                                               action  
clean_actions                                      prediction    label                                 
Read rules suspended read second and ordered to... uncategorized multiread_second_order_third       1  
                                                                 read_second_order_third           92  
Read second and ordered to a third reading         uncategorized read_second_order_third          702  
Ordered to a third reading                         uncategorized unc                              166  
Read second rules suspended ordered to a third ... engrossment   unc                                1  
Read second ordered to a third reading rules su... engrossment   unc                               19  
Read second ordered to a third reading rules su... engrossment   unc                                1  
Read second ordered to a t

,action_id,action,prediction,clean_actions
label,,,,
Emergency_preamble_adopted,2,2,2,2
adopted,366,366,366,366
amended,81,81,81,81
amendment,1214,1214,1214,1214
amendments_considered,36,36,36,36
enacted,249,249,249,249
insist_on_amendment,22,22,22,22
multiread_second_order_third,1,1,1,1
non_concur,28,28,28,28
